# Check Organization Readiness

This notebook will look at the configuration of the organization and identity and gaps in the configuration.

This notebook does make any changes to the configuration, however will write some parameter store values to simplify the configuration of the notebooks.

## Accounts:
* **Management Account**: The root account that owns the AWS organization. Typically resources are note deployed to the root account, but some configurations must be done in this account. For example, the cloudtrail organizational trail is created in the management account.
* **Logging**: The account that holds the log files. Typically this account contains the buckets for the CloudTrail and other logs, and Athena tables used to query them.
* **CloudFormation StackSet Delegated Account**: An account designated by the management account to own stacksets.


In [2]:
from jupyterirtools import sso
import os
import time
import importlib
import json

sso.login("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=RQVG-TCWN to activate the session


<IPython.core.display.Javascript object>

In [4]:
import boto3 
from IPython.display import display, Markdown

found_org_trail = False
cloudtrail_client = boto3.client('cloudtrail', region_name = "us-east-1")

paginator = cloudtrail_client.get_paginator('list_trails')

response_iterator = paginator.paginate()
for page in response_iterator:
    for trail in page["Trails"]:
        trail_response = cloudtrail_client.get_trail(Name=trail['TrailARN'])
        if trail_response['Trail']['IsOrganizationTrail'] and trail_response['Trail']['IsMultiRegionTrail']:
            found_org_trail = True
            org_trail_info = trail_response['Trail']
            org_trail_arn = org_trail_info['TrailARN']

#if not found_org_trail:
if not found_org_trail:
    display(Markdown("""
**Organizational Cloudtrail not found**: Please configure an 
organizational trail with the following settings and rerun this noteboot to verify.
There are three options for configuring the trail:
* If you are using Control Tower, configure the trail in the [landing zone](https://docs.aws.amazon.com/controltower/latest/userguide/cloudtrail.html).
* Configure the Organization trail [manually](https://docs.aws.amazon.com/awscloudtrail/latest/userguide/creating-trail-organization.html).
* Use the [Configure Organizational Trail](configuration-notebooks/config_org_trail.ipynb) notebook."""))
else:
    trail_dest = f"s3://{org_trail_info['S3BucketName']}/{org_trail_info.get('S3KeyPrefix','')}"
    session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['LOGGING_ACCOUNT'])
    ssm_client = session.client('ssm')
    aggregator_param_response = ssm_client.put_parameter(Name='Jupyter-CloudTrail-Destination', Type='String', Overwrite=True, Value=trail_dest)

    display(Markdown(f"""
**Organizational Cloudtrail found**

Stored the destination of the trail in the Systems Manager Parameter **Jupyter-CloudTrail-Destination** in account {os.environ['LOGGING_ACCOUNT']} to the value:
{trail_dest}
"""))
 

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)



**Organizational Cloudtrail found**

Stored the destination of the trail in the Systems Manager Parameter **Jupyter-CloudTrail-Destination** in account 251344881676 to the value:
s3://organization-logging-buck-cfncloudtraillogsbucket-3jqplzdobirg/


# Get All Accounts and Regions in Org



In [ ]:
accounts = []

org_client = boto3.client('organizations')

paginator = org_client.get_paginator('list_accounts')

page_iterator = paginator.paginate()
for page in page_iterator:
    for account in page['Accounts']:
        accounts.append(account['Id'])

ec2_client = boto3.client('ec2')
regions = []

region_response = ec2_client.describe_regions()

for region in region_response['Regions']:
    regions.append(region['RegionName'])


# Check AWS Config across the organization
AWS Config provides configuration management services to enforce rules and provide a simple interface to query resources across the organization. The following script will check all the accounts and regions in the org and validate that AWS config is enabled.

In [ ]:
missing_regions = 0

for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in regions:
        
        session = sso.get_session("Jupyter-IR-AdministratorAccess", account, region_name)
        config_client = session.client('config')
        delivery_channels_result = config_client.describe_delivery_channels()
        delivery_channel_found = False
        for delivery_channel in delivery_channels_result['DeliveryChannels']:
            delivery_channel_found = True
        
        if not delivery_channel_found:
            missing_regions += 1
            print(f"Delivery Channel NOT found in account {account} in region {region_name}")

print(f"{missing_regions} regions not configured with an AWS Config delivery channel")

### Next Steps

If any regions are not configured with a delivery channel, remediate with:
* Follow the instructions in [AWS Config Best Practices](https://aws.amazon.com/blogs/mt/aws-config-best-practices/#:~:text=You%20can%20use%20AWS%20CloudFormation,be%20recorded%20in%20AWS%20Config) to enable all accounts and regions.


## Check AWS Config Aggregators
AWS Config Aggregators share AWS config information across the organization so all the information is pooled across all accounts and all regions. This allows you to run queries and compliance dashboards across the entire organization.

In [ ]:
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])
config_client = session.client('config')

describe_configuration_aggregators_response = config_client.describe_configuration_aggregators()

found_org_all_region_aggregator = False
config_aggregator_name = ""

for aggregator in describe_configuration_aggregators_response['ConfigurationAggregators']:
    if 'OrganizationAggregationSource' in aggregator and aggregator['OrganizationAggregationSource']['AllAwsRegions']:
        found_org_all_region_aggregator = True
        config_aggregator_name = aggregator['ConfigurationAggregatorName']
        
if found_org_all_region_aggregator:

    session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])
    ssm_client = session.client('ssm')
    aggregator_param_response = ssm_client.put_parameter(Name='Jupyter-Config-Aggregator', Type='String', Overwrite=True, Value=config_aggregator_name)

    display(Markdown(f"""
**Organizational AWS Config Aggregator found**

Stored the destination of the trail in the Systems Manager Parameter **Jupyter-Config-Aggregator** in account {os.environ['MANAGEMENT_ACCOUNT']} to the value:
{config_aggregator_name}
"""))
    
else:
    display(Markdown("""
**No Organaization-wide aggregator found**: Please configure an 
organizational AWS Config aggregator that covers all accounts and all region. Options for setting one up:
* Configure the Organization aggregator [manually](https://aws.amazon.com/blogs/mt/org-aggregator-delegated-admin/).
* Use the [Configure Organizational AWS Config Aggregator](configuration-notebooks/config_org_aggregator.ipynb) notebook."""))


# Check Flow Logs

This section will check the configuration of flow logs across the AWS Organization. Having vpcs without flow logs is a risk, even for those in sandbox and non-production accounts, as the organization will lose visibility into potential vulnerbility.

The notebooks will query flow logs via Athena, as it presents a convenient way to query across accounts and regions. If the flow logs write to cloudwatch, an export task should be configured to write the logs to S3.

For those written to S3, they should all write to a central log to faciliate querying accross the organization.



In [ ]:
no_flow_logs = []
cloudwatch_flow_logs = []
s3_log_destinations = []
for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in regions:
        session = sso.get_session("Jupyter-IR-AdministratorAccess", account, region_name)
        client = session.client('ec2')
        response = client.describe_vpcs()
        for vpc in response['Vpcs']:
            flow_log_response = client.describe_flow_logs( Filters=[
                    {
                        'Name': 'resource-id',
                        'Values': [ vpc['VpcId'] ]
                    }])
            if len(flow_log_response['FlowLogs']) == 0:
                no_flow_logs.append(f"{vpc['VpcId']:20}\t{account}\t{region_name}")
            else:
                flow_log_info = flow_log_response['FlowLogs'][0]
                if flow_log_info['LogDestinationType'] == "cloud-watch-logs":
                    cloudwatch_flow_logs.append(f"{vpc['VpcId']:20}\t{account}\t{region_name}")
                elif flow_log_info['LogDestinationType'] == "s3":
                    log_destination = flow_log_info['LogDestination'][13:]
                    s3_log_destinations.append(f"{vpc['VpcId']:20}\t{account}\t{region_name:14}\t{log_destination}")

print("\nVPCs without Flow Logs")
print(f"{'resource-id':20}\t{'Account ID':12}\t{'Region'}")
print(f"{'-'*20}\t{'-'*12}\t{'-'*10}")
for row in no_flow_logs:
    print(row)

print("\nVPCs with Flow Logs sent to Cloudwatch logs")
print(f"{'resource-id':20}\t{'Account ID':12}\t{'Region'}")
print(f"{'-'*20}\t{'-'*12}\t{'-'*10}")
for row in cloudwatch_flow_logs:
    print(row)
    
print("\nVPCs with Flow Logs sent to S3")
print(f"{'resource-id':20}\t{'Account ID':12}\t{'Region':14}\t{'S3 Location'}")
print(f"{'-'*20}\t{'-'*12}\t{'-'*14}\t{'-'*14}")
for row in s3_log_destinations:
    print(row)

### Next Steps

**Recommendations**
* Enable flow logs on all VPCs
* Centralize Logging in a single S3 bucket.

**Remediation**
* **Option 1**: [Manually configure flow logs](https://docs.aws.amazon.com/vpc/latest/userguide/flow-logs.html)
* **Option 2**: Execute the runbooks:
  * [Remove All Flowlogs](configuration-notebooks/remove_flow_logs.ipynb) **WARNING: This will remove your current flow log configurations**
  * [Auto Enable Flowlogs](configuration-notebooks/auto_enable_flowlogs.ipynb) Will use AWS Config auto remediation to automatically configure flow logs across the AWS Organization.

# Check Athena Configuration

The section will validate the Athena tables are configured and ready to access.

In [6]:
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['LOGGING_ACCOUNT'])
ssm_client = session.client('ssm')
glue_client = session.client('glue')

database_name=""
cloudtrail_table=""
flow_logs_table=""

config_error = False

try:
    database_response = ssm_client.get_parameter(Name='Jupyter-Athena-Glue-Database')
    database_name = database_response['Parameter']['Value']
except ssm_client.exceptions.ParameterNotFound:
    print("Jupyter-Athena-Glue-Database not found")
    config_error = True
try:
    cloudtrail_response = ssm_client.get_parameter(Name='Jupyter-Athena-CloudTrail-Table')
    cloudtrail_table = cloudtrail_response['Parameter']['Value']
except ssm_client.exceptions.ParameterNotFound:
    print("Jupyter-Athena-CloudTrail-Table not found")
    config_error = True

try:
    flowlog_response = ssm_client.get_parameter(Name='Jupyter-Athena-FlowLogs-Table')
    flow_logs_table = flowlog_response['Parameter']['Value']
except ssm_client.exceptions.ParameterNotFound:
    print("Jupyter-Athena-FlowLogs-Table not found")
    config_error = True

if database_name != "" and cloudtrail_table != "":
    try:
        cloudtrail_response = glue_client.get_table(
            DatabaseName=database_name,
            Name=cloudtrail_table
        )
    except glue_client.exceptions.EntityNotFoundException:
        config_error = True
        print(f"Glue: Cloudtrail {cloudtrail_table} not found")
        
if database_name != "" and flow_logs_table != "":
    try:
        flowlog_response = glue_client.get_table(
            DatabaseName=database_name,
            Name=flow_logs_table
        )    
    except glue_client.exceptions.EntityNotFoundException:
        config_error = True
        print(f"Glue: Flow Logs {flow_logs_table} not found")    
        
if config_error:
        display(Markdown("""

### Configure CloudTrail and Flowlogs:
* Option 1:
  * Set up [Athena cloud trail](https://docs.aws.amazon.com/athena/latest/ug/cloudtrail-logs.html) in the console.
  * Set the Jupyter-Athena-CloudTrail-Table Systems Manager parameter in the logging account.
  * Set up [Athena flow logs](https://docs.aws.amazon.com/athena/latest/ug/vpc-flow-logs.html) in the console.
  * Set the Jupyter-Athena-FlowLogs-Table Systems Manager parameter in the logging account.
* Option 2: 
  * Use the [Configure Athena CloudTrail](configuration-notebooks/config_athena_cloudtrail.ipynb) notebook.
"""))
else:
     display(Markdown("""
     Athena configuration successful.
"""))


     Athena configuration successful.


In [9]:
from jupyterirtools import athena
import importlib
importlib.reload(athena)

sql = f"""SELECT * 
FROM "${{database_name}}"."${{cloudtrail_table}}" 
WHERE accountId = '{os.environ['MANAGEMENT_ACCOUNT']}'
limit 10;"""

display(athena.run_query(sql))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


,eventversion,useridentity,eventtime,eventsource,eventname,awsregion,sourceipaddress,useragent,errorcode,errormessage,...,eventtype,apiversion,readonly,recipientaccountid,serviceeventdetails,sharedeventid,vpcendpointid,accountid,region,timestamp
0,1.08,"{type=AssumedRole, principalid=AROAIAQ76LM7Q5B...",2022-11-03T04:29:32Z,ssm.amazonaws.com,UpdateInstanceInformation,us-east-2,18.221.114.64,aws-sdk-go/1.41.4 (go1.18.3; linux; amd64) ama...,None,None,...,AwsApiCall,None,false,383086473915,None,None,None,383086473915,us-east-2,2022/11/03
1,1.08,"{type=AssumedRole, principalid=AROAJTOZRHM2SUE...",2022-11-03T18:19:47Z,s3.amazonaws.com,GetObject,us-east-2,cloudformation.amazonaws.com,cloudformation.amazonaws.com,AccessDenied,Access Denied,...,AwsApiCall,None,true,383086473915,None,ef36970b-379a-431c-b2ad-701d494a5170,vpce-16a4477f,383086473915,us-east-2,2022/11/03
2,1.08,"{type=AssumedRole, principalid=AROAVSMN3WK5UHW...",2022-11-03T08:08:38Z,s3.amazonaws.com,PutObject,us-west-2,config.amazonaws.com,config.amazonaws.com,AccessDenied,Access Denied,...,AwsApiCall,None,false,383086473915,None,bf381369-2063-4400-8f7b-fc34995cc5fe,None,383086473915,us-west-2,2022/11/03
3,1.08,"{type=AssumedRole, principalid=AROAJTOZRHM2SUE...",2022-11-03T18:10:12Z,s3.amazonaws.com,GetObject,eu-north-1,cloudformation.amazonaws.com,cloudformation.amazonaws.com,AccessDenied,Access Denied,...,AwsApiCall,None,true,383086473915,None,3bde8913-e89a-4bfb-b861-aa6cfa6345e5,vpce-0b38230eb5e426fb0,383086473915,eu-north-1,2022/11/03
4,1.08,"{type=AssumedRole, principalid=AROAJURR22XJKNX...",2022-11-03T05:43:28Z,kms.amazonaws.com,Decrypt,us-east-1,codepipeline.amazonaws.com,codepipeline.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-east-1,2022/11/03
5,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-11-03T05:46:17Z,sts.amazonaws.com,AssumeRole,us-east-1,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,1d8abfd4-f476-4192-b7c2-40ae2a320599,None,383086473915,us-east-1,2022/11/03
6,1.08,"{type=AssumedRole, principalid=AROAIAQ76LM7Q5B...",2022-11-03T17:19:32Z,ssm.amazonaws.com,UpdateInstanceInformation,us-east-2,18.221.114.64,aws-sdk-go/1.41.4 (go1.18.3; linux; amd64) ama...,None,None,...,AwsApiCall,None,false,383086473915,None,None,None,383086473915,us-east-2,2022/11/03
7,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-11-03T17:20:41Z,kms.amazonaws.com,GenerateDataKey,us-east-2,rds.amazonaws.com,rds.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,686dd909-8061-45f4-9121-624f4e690fa3,None,383086473915,us-east-2,2022/11/03
8,1.08,"{type=AssumedRole, principalid=AROAIAQ76LM7Q5B...",2022-11-03T17:23:24Z,ssm.amazonaws.com,ListInstanceAssociations,us-east-2,18.221.114.64,aws-sdk-go/1.41.4 (go1.18.3; linux; amd64) ama...,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-east-2,2022/11/03
9,1.08,"{type=AssumedRole, principalid=AROAIAQ76LM7Q5B...",2022-11-03T17:24:32Z,ssm.amazonaws.com,UpdateInstanceInformation,us-east-2,18.221.114.64,aws-sdk-go/1.41.4 (go1.18.3; linux; amd64) ama...,None,None,...,AwsApiCall,None,false,383086473915,None,None,None,383086473915,us-east-2,2022/11/03


In [10]:
from jupyterirtools import athena
import importlib
importlib.reload(athena)

display(athena.get_vpc_flow_by_account(os.environ['MANAGEMENT_ACCOUNT']))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


OperationalError: Permission denied on S3 path: s3://flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0/AWSLogs/383086473915/vpcflowlogs/us-east-2/2022/11/03/383086473915_vpcflowlogs_us-east-2_fl-04986d7dfc5d30246_20221103T2015Z_d6c2d5c6.log.parquet